In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor

# 1) 소스별 정제
pub = pd.read_csv('data/Pubchem_ASK1.csv')
chem = pd.read_csv('data/ChEMBL_ASK1(IC50).csv', sep=';', low_memory=False)

# --- PubChem: IC50 + '=' 만 사용, 단위가 있으면 nM만 남김 ---
smiles_col_pub = 'SMILES' if 'SMILES' in pub.columns else 'Smiles'
pub = pub.rename(columns={smiles_col_pub:'SMILES'})
if 'Activity_Type' in pub.columns:
    pub = pub[pub['Activity_Type'].str.upper().eq('IC50')]
if 'Activity_Qualifier' in pub.columns:
    pub = pub[pub['Activity_Qualifier'].isin(['=', '~'])]
# 단위 컬럼이 있다면 nM만 남김 (없으면 스킵)
for c in ['Units','Activity_Units','Standard_Units','Standard Units']:
    if c in pub.columns:
        pub = pub[pub[c].str.lower().eq('nm')]
        break
pub = pub[['SMILES','Activity_Value']].dropna()
pub = pub.rename(columns={'Activity_Value':'IC50_nM'})

# --- ChEMBL: IC50 + nM + '=' 만 사용 ---
chem = chem.rename(columns={'Smiles':'SMILES'})
# 컬럼명 케이스/스페이스 차이를 넉넉히 처리
def has(col): 
    return col in chem.columns
stype = 'Standard Type' if has('Standard Type') else ('Standard_Type' if has('Standard_Type') else None)
srel  = 'Standard Relation' if has('Standard Relation') else ('Standard_Relation' if has('Standard_Relation') else None)
sunit = 'Standard Units' if has('Standard Units') else ('Standard_Units' if has('Standard_Units') else None)
sval  = 'Standard Value' if has('Standard Value') else ('Standard_Value' if has('Standard_Value') else ('Value' if has('Value') else None))

if stype: chem = chem[chem[stype].str.upper().eq('IC50')]
if srel:  chem = chem[chem[srel].isin(['=', '~'])]
if sunit: chem = chem[chem[sunit].str.lower().eq('nm')]
chem = chem[['SMILES', sval]].dropna().rename(columns={sval:'IC50_nM'})

# 2) 숫자 변환 + 유효 범위 필터
for df in (pub, chem):
    df['SMILES'] = df['SMILES'].astype(str).str.strip()
    df['IC50_nM'] = pd.to_numeric(df['IC50_nM'], errors='coerce')

pub  = pub.dropna(subset=['IC50_nM'])
chem = chem.dropna(subset=['IC50_nM'])

# 0 또는 음수 제거, 극단치 클리핑(선택)
pub  = pub[pub['IC50_nM'] > 0]
chem = chem[chem['IC50_nM'] > 0]
upper = 1e6  # 필요시 조정
pub.loc[pub['IC50_nM']>upper,  'IC50_nM'] = upper
chem.loc[chem['IC50_nM']>upper,'IC50_nM'] = upper

# 3) 합치고, 같은 SMILES 여러 값이면 log 영역 중앙값 집계
train = pd.concat([pub, chem], ignore_index=True)
train = train.dropna(subset=['SMILES','IC50_nM'])
train['log10_IC50'] = np.log10(train['IC50_nM'])

# SMILES 단위로 중앙값(권장) 집계
train = (train
         .groupby('SMILES', as_index=False)
         .agg(log10_IC50=('log10_IC50','median')))

# 4) 피처: Morgan FP
def mol_from_smiles(s):
    m = Chem.MolFromSmiles(s)
    return m

train['mol'] = train['SMILES'].apply(mol_from_smiles)
train = train[train['mol'].notnull()].reset_index(drop=True)

def mfp(m, radius=2, n_bits=2048):
    return np.array(AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits), dtype=np.uint8)

X = np.vstack(train['mol'].apply(mfp).values)  # (N, 2048)
y_log = train['log10_IC50'].values             # log10(nM) 라벨

# 5) 학습/검증 분리
X_tr, X_te, y_tr, y_te = train_test_split(X, y_log, test_size=0.3, random_state=42)

# 6) 모델 학습 (log 스케일에서)
model = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)
model.fit(X_tr, y_tr)
y_pred_log = model.predict(X_te)

# 7) 평가: 로그/원스케일 둘 다 확인
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 로그 스케일
mae_log = mean_absolute_error(y_te, y_pred_log)
# rmse_log = mean_squared_error(y_te, y_pred_log, squared=False)
r2_log = r2_score(y_te, y_pred_log)

# 원 스케일(nM)로 변환해서도 보기
y_te_nm    = 10**y_te
y_pred_nm  = 10**y_pred_log
mae_nm  = mean_absolute_error(y_te_nm, y_pred_nm)
# rmse_nm = mean_squared_error(y_te_nm, y_pred_nm, squared=False)
r2_nm   = r2_score(y_te_nm, y_pred_nm)

print(f"[log10(nM)]  MAE={mae_log:.3f}  R2={r2_log:.3f}")
print(f"[nM]         MAE={mae_nm:.3f}   R2={r2_nm:.3f}")

/var/folders/fx/bdj1bhw11037xfwjzgrx6vc80000gn/T/ipykernel_6705/1768226518.py:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  pub = pd.read_csv('data/Pubchem_ASK1.csv')


[log10(nM)]  MAE=0.510  R2=0.553
[nM]         MAE=8.351   R2=-0.011
